<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [11]:
%run "../../xUtilz/utilz_only_loader.ipynb"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic


In [12]:
db = pd.read_csv( "../../datasets/LG/IN/181109_Referral_Reasons.csv" )

db["CHV Name"] = "CHV " +  pd.Series( pd.factorize( db["CHV Name"])[0] + 1).astype( str)
db["Patient Name"] = "Patient " +  pd.Series( pd.factorize( db["Patient Name"])[0] + 1).astype( str)
db.drop( "CHV Phone", axis=1, inplace=True)

var_Category_Referral_Reason = "Category Referral Reason"
var_Unsupported = "Unsupported Cases"
var_HIVST = "HIVST"
var_Display_Colz = ["Date of referral", "CHV Name", "Patient Name" ,"Referral Reason", "Reason for Referral"]


reasons_groupz = { 
    "Neonates" : [ "months", "umbilical", "umbllical"], 
    "Pregnancy" : ["delivery"], 
    "U5 Danger Signs" : [ "Danger", "Diarrhoea", "Diarrhea", "Dairrhea", "Coughing", "Cough", "Fever","Pneumonia", "Pnuemonia",
                        "Pheumonia", ],
    "HIVST" : ["reactive", "partner", "intimate"], 
    "Unsupported Cases" : []
}

db[ var_Category_Referral_Reason ] = db["Referral Reason"].apply( lambda x: searchString(x, reasons_groupz) ) 

# 1. set Date of referral to a datetime type 
db["Date of referral"] = pd.to_datetime( db["Date of referral"], format="%d-%b-%y") # inplace=True)

# 2. Extract Month and year categories 
db["Year"] = db["Date of referral"].dt.year 
db["Month"] = db["Date of referral"].dt.strftime('%b-%y') 
db["Day_of_Week"] = db["Date of referral"].dt.weekday_name.str[:3] 

db.sort_values( by='Date of referral', inplace=True)

LAST_UPDATED = db["Date of referral"].max().strftime( '%d-%b-%Y')

In [30]:
varALLPlaceholder = "All"
var_FilterOptions = [ varALLPlaceholder ]+db["Category Referral Reason"].unique().tolist()

var_TableFields = ["Date of referral", "CHV Name", "Patient Name", "Category Referral Reason", "Referral Reason" ]

var_FilterOptions

#def updateDataset( x=varALLPlaceholder): 
#    return db[ db['Category Referral Reason'] == x][var_TableFields] if x != varALLPlaceholder else db[var_TableFields]

#interact( updateDataset, 
#         x=[ varALLPlaceholder ]+db["Category Referral Reason"].unique().tolist(), )

['All',
 'Unsupported Cases',
 'U5 Danger Signs',
 'Pregnancy',
 'No Response',
 'Neonates',
 'HIVST']

In [4]:
app = dash.Dash( "hello world")
app.layout = dhtml.Div(
[
    dcc.Dropdown(
        id='referral_cats',
        options = var_FilterOptions,
        value = varALLPlaceholder
    ),
    dcc.Graph( id='dbTable')
], style={
    'width' : '500'
})

@app.callback( Output('dbTable', 'figure'), [Input('referral_cats', 'value')])
def updateTable( selected_value):
    df = dweb.DataReader(
        selected_value,
        'google', 
        dt( 2017, 1, 1),
        dt.now()
    )
    return{
        'data': [
            {
                'x' : df.index,
                'y' : df.Close
            }
        ],
        'layout' : { 'margin': { 'l': 40, 'r': 0, 't': 20, 'b': 30 }}
    }

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

#app.run_server()
    

In [16]:
## https://dash.plot.ly/getting-started 

def genTable( db, max_r=10):
    return dhtml.Table(
        #Header
        [ dhtml.Tr( [ dhtml.Th( col) for col in db.columns])] + 

        # boddy
        [ dhtml.Tr([
            dhtml.Td( db.iloc[i][col]) for col in db.columns ]) for i in range( min(len(db), max_r))
        ]
    ) 


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

collors = {
    'bg': '#111111',
    'txt': '#7FDBFF'
}


app = dash.Dash( __name__, external_stylesheets=external_stylesheets)

app.layout = dhtml.Div(
style= {'backgroundColor': collors['bg'], 'textAlign': 'center', 'color': collors['txt']}, 
children=[
    dhtml.H1( children='Hello Dash'),
    
    dhtml.Div( children= '''
    Dash: A web application framework for Python
    '''),
    
    dcc.Graph(
        id='graph1',
        figure={
            'data' : [
                { 'x': [ 1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'}, 
                { 'x' : [1, 2, 3], 'y': [2, 3, 5], 'type': 'bar', 'name': 'MTR'}
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }     
    ),
    
    
    dhtml.Div( [
        dhtml.H4( "Table Title Here"), 
        genTable( db )
    ]) 
])



app.run_server( )

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2018 16:51:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 16:51:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 16:51:25] "GET /_dash-dependencies HTTP/1.1" 200 -


In [20]:
import plotly.graph_objs as go 

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/' +
    '5d1ea79569ed194d432e56108a04d188/raw/' +
    'a9f9e8076b837d541398e999dcbac2b2826a81f8/'+
    'gdp-life-exp-2007.csv')


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dhtml.Div([
    dcc.Graph(
        id='graph3',
        figure={
            'data': [
                go.Scatter(
                    x=df[df['continent'] == i ][ 'gdp per capita' ], 
                    y=df[ df['continent'] == i ][ 'life expectancy'], 
                    text=df[ df['continent']==i]['country'],
                    mode='markers', 
                    opacity= 0.7, 
                    marker= {
                        'size': 15,
                        'line': { 'width': 0.5, 'color': 'white'}
                    },
                    name= i 
                ) for i in df. continent.unique() 
                
            ], 
            'layout': go.Layout(
                xaxis={'type': 'log', 'title': 'GDP Per Capita' }, 
                yaxis={ 'title': "Life Expectancy"}, 
                margin={ 'l': 40, 'b': 40, 't': 10, 'r': 10},
                legend={ 'x': 0, 'y': 1},
                hovermode= 'closest'
            )
        }
    )
])

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2018 17:02:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 17:02:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 17:02:06] "GET /_dash-dependencies HTTP/1.1" 200 -


In [21]:
## And Now in Markdown instead of html_components
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

markdwon_text = '''
### Dash and Markdown

The quick brown fox jumped over the lazy dogs. 

Something something [that links here](http://www.google.com)

And some listing too
1. quick  
2. brown 
3. lazy 

'''

app.layout = dhtml.Div([
    dcc.Markdown( children=markdwon_text)
])

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2018 17:10:56] "GET / HTTP/1.1" 200 -


In [37]:
## Components << Not showing list options + check box is selecting everything

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dhtml.Div([
    dhtml.Label('DropDown'),
    dcc.Dropdown(
        options = var_FilterOptions,
        value = varALLPlaceholder
    ),
    
    dhtml.Label( "Multi-Select Dropdown"),
    dcc.Dropdown(
        options = { 'label': "NYC"},
        value = varALLPlaceholder,
        multi=True
    ),
    
    dhtml.Label('Radio Items'),
    dcc.RadioItems(
        options=[
            {'label': 'New York City', 'value': 'NYC'},
            {'label': u'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'}
        ],
        value='MTL'
    ),
    
    dhtml.Label( "Checkboxes"),
    dcc.Checklist(
        options = var_FilterOptions + [
            {'label': 'New York City', 'value': 'NYC'},
            {'label': u'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'} ], 
        values = [varALLPlaceholder, "HIVST"] 
    ),
    
    
    dhtml.Label( "Text Input"),
    dcc.Input(
        type='text',
        value = varALLPlaceholder,
    ),
    
    
    dhtml.Label( "Slider"),
    dcc.Slider(
        min=0,
        max=9,
        marks={ i: 'Label {}'.format(i) if i == 1 else str(i) for i in range(1,6 )}, 
        value=5
    )
], style={ 'columnCount': 2})

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2018 17:26:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 17:26:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 17:26:23] "GET /_dash-dependencies HTTP/1.1" 200 -


In [41]:
### Now with Call backs

app = dash.Dash( __name__, external_stylesheets=external_stylesheets)

app.layout = dhtml.Div([
    dcc.Input( id='tl1', type='text', value='enter category'),
    dhtml.Div( id = 'td1', children='Hellow World!!!' )
])

@app.callback(
    Output( component_id='td1', component_property='children'), 
    [Input( component_id='tl1', component_property='value')]
)
def update_output_div( input_value):
    return 'You entered "{ }"'.format( input_value)


app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
[2018-11-09 17:36:00,287] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File 

127.0.0.1 - - [09/Nov/2018 17:36:05] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:36:05,301] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

127.0.0.1 - - [09/Nov/2018 17:36:07] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:36:07,857] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

127.0.0.1 - - [09/Nov/2018 17:36:09] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:36:09,246] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

127.0.0.1 - - [09/Nov/2018 17:36:10] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:36:10,877] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

127.0.0.1 - - [09/Nov/2018 17:36:13] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:36:18,689] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

In [ ]:
## Now inside of Jupyter

from IPython import display as disp2

def show_app( app, port=9999, width=700, height=350, offline=False, in_binder=None):
    url = "http://localhost:%d"%port 
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, width=width, height=height) 
    disp2.display_html( iframe, raw=True)
    return app.run_server(debug=False, host='0.0.0.0', port=port)

                                                                                                                                          
show_app( app)

Open in new window

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2018 17:44:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 17:44:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2018 17:44:02] "GET /_dash-dependencies HTTP/1.1" 200 -
[2018-11-09 17:44:02,480] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_

127.0.0.1 - - [09/Nov/2018 17:44:07] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:44:07,806] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

127.0.0.1 - - [09/Nov/2018 17:44:08] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:44:08,965] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

127.0.0.1 - - [09/Nov/2018 17:44:11] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:44:11,264] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](*

KeyError: ' '
127.0.0.1 - - [09/Nov/2018 17:44:13] "POST /_dash-update-component HTTP/1.1" 500 -
[2018-11-09 17:44:13,254] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\BILHA\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[ru